In [2]:
from langchain.utilities import GoogleSerperAPIWrapper, BashProcess
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain, LLMMathChain


In [3]:
search = GoogleSerperAPIWrapper()
llm = OpenAI(temperature=0)


In [4]:
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

In [5]:
bash = BashProcess()

In [6]:
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="Bash",
        func=bash.run,
        description="useful for when you need to execute bash commands"
    ),
]


In [7]:
prefix = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:"""
suffix = """Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

print(prompt.template)



Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

Search: useful for when you need to answer questions about current events
Calculator: useful for when you need to answer questions about math
Bash: useful for when you need to execute bash commands

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator, Bash]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}


In [8]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, max_steps=30)


In [9]:
agent_executor.run("What is the diffrent in price of latest apple phone model compare to the latest samsung phone model?")



> Entering new AgentExecutor chain...
Thought: I need to compare the prices of the two phones
Action: Search
Action Input: Latest apple phone model price vs latest samsung phone model price
Observation: The iPhone 14 and Galaxy S23 both start at around $799, but Samsung's higher-end phones are more expensive. The S23 Plus costs $999.99, which is $100 more than Apple's iPhone 14 Plus, while the S23 Ultra has an MSRP of $1,199.99 — $200 more than the iPhone 14 Pro and $100 more than the iPhone 14 Pro Max. Feb 2, 2023
Thought: I now know the final answer
Final Answer: Arrr, it be true that the latest Samsung phone model be costin' more than the latest Apple phone model.

> Finished chain.


"Arrr, it be true that the latest Samsung phone model be costin' more than the latest Apple phone model."